# DB connection 

In [1]:
from pymongo import MongoClient
import matplotlib.pyplot as plt

client = MongoClient() # we run on localhost and default port
db = client['spotify']

# Get Collections

In [28]:
features = db['features']
tracks = db['tracks']
artists = db['artists']
albums = db['albums']
all = db['all']

In [29]:
print(features) 
print(tracks) 
print(artists) 
print(albums) 
print(all)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'features')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'tracks')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'artists')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'albums')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'spotify'), 'all')


In [30]:
def display_results(res):
    for entry in res:
        print(entry)

# Query 11
### Average track length per genre

Unfortunately, the "genre" field is related to the artist only, not to the album/track, which would be more accurate.

(By the way, how many genres do we have?)

In [5]:
uniqueGenresQuery = [
  {
    '$group': {
      '_id': "$genre_0",
    }
  },
  {
    '$group': {
      '_id': True,
      'uniqueGenres': {'$sum': 1}
    }
  } 
]

result_uniqueGenresQuery = list(all.aggregate(uniqueGenresQuery))

In [6]:
result_uniqueGenresQuery

[{'_id': True, 'uniqueGenres': 3009}]

In [7]:
query11 = [
  {
    '$group': {
      '_id': "$genre_0",
      'averageTrackLengthPerGenre': {'$avg': "$duration_ms"}
    }
  },
  {
    '$project': {
      '_id': 0,
      'genre': "$_id",
      'averageTrackLengthPerGenre': 1
    }
  },
  {
    '$facet': {
        "shortest100": [
            { '$sort': { "averageTrackLengthPerGenre": 1 } },
            { '$limit': 100 }
        ],
        "longest100": [
            { '$sort': { "averageTrackLengthPerGenre": -1 } },
            { '$limit': 100 }
        ]
    }
  }
]

result_query11 = list(all.aggregate(query11))

In [8]:
result_query11

[{'shortest100': [{'averageTrackLengthPerGenre': 7882.4,
    'genre': 'sound effects'},
   {'averageTrackLengthPerGenre': 29568.317073170732, 'genre': 'ringtone'},
   {'averageTrackLengthPerGenre': 59256.0, 'genre': 'cruise'},
   {'averageTrackLengthPerGenre': 61558.42857142857, 'genre': 'birthday'},
   {'averageTrackLengthPerGenre': 66968.65384615384, 'genre': 'hauntology'},
   {'averageTrackLengthPerGenre': 68000.0, 'genre': 'deep dance pop'},
   {'averageTrackLengthPerGenre': 69782.04761904762,
    'genre': 'nordic soundtrack'},
   {'averageTrackLengthPerGenre': 74288.0, 'genre': 'russian viral rap'},
   {'averageTrackLengthPerGenre': 75696.0, 'genre': 'japanoise'},
   {'averageTrackLengthPerGenre': 79004.0, 'genre': 'hypnagogic pop'},
   {'averageTrackLengthPerGenre': 80184.6, 'genre': 'plena uruguaya'},
   {'averageTrackLengthPerGenre': 82726.1, 'genre': 'bachchon ke geet'},
   {'averageTrackLengthPerGenre': 83465.11627906977,
    'genre': 'german romanticism'},
   {'averageTrackL

Funnily enough, there exist genres like "sound effects" and "ringtone".
Anyway, it seems like the shortest tracks come from genres like punk or are related to "virality" at first sight, while the longest ones come from classical music.
Let's explore deeper!

In [9]:
shortest100 = [entry['shortest100'] for entry in result_query11]
shortest100 = shortest100[0]
longest100 = [entry['longest100'] for entry in result_query11]
longest100 = longest100[0]
shortest_genres = [entry['genre'] for entry in shortest100]
longest_genres = [entry['genre'] for entry in longest100]

# hardcore_count = 0
# classical_count = 0

# hardcore_count = sum(s.lower().split().count('hardcore') for s in shortest_genres)
# classical_count = sum(s.lower().split().count('classical') for s in longest_genres)

# print(hardcore_count)
# print(classical_count)

In [ ]:
shortest_genres

In [ ]:
longest_genres

In [10]:
from collections import Counter

# Concatenate all strings into a single space-separated string
shortest_genres_text = ' '.join(shortest_genres)

# Split the combined string into a list of words
shortest_genres_words = shortest_genres_text.lower().split()

# Use Counter to count the frequency of each word
shortest_genres_word_frequency = Counter(shortest_genres_words)

# Print the words ordered by frequencies in descending order
for word, frequency in shortest_genres_word_frequency.most_common():
    if frequency >= 3:
        print(f'The word "{word}" appears {frequency} times.')
        
print('')

# Concatenate all strings into a single space-separated string
longest_genres_text = ' '.join(longest_genres)

# Split the combined string into a list of words
longest_genres_words = longest_genres_text.lower().split()

# Use Counter to count the frequency of each word
longest_genres_word_frequency = Counter(longest_genres_words)

# Print the words ordered by frequencies in descending order
for word, frequency in longest_genres_word_frequency.most_common():
    if frequency >= 3:
        print(f'The word "{word}" appears {frequency} times.')

# If you want to get the frequency of a specific word, you can do
# specified_word = "hardcore"
# specified_word_frequency = word_frequency[specified_word.lower()]
# print(f'The word "{specified_word}" appears {specified_word_frequency}

The word "indie" appears 8 times.
The word "rap" appears 7 times.
The word "pop" appears 5 times.
The word "hip" appears 5 times.
The word "hop" appears 5 times.
The word "russian" appears 4 times.
The word "hardcore" appears 4 times.
The word "musica" appears 4 times.
The word "music" appears 4 times.
The word "german" appears 3 times.
The word "beats" appears 3 times.
The word "japanese" appears 3 times.
The word "phonk" appears 3 times.
The word "children's" appears 3 times.
The word "kazakh" appears 3 times.
The word "punk" appears 3 times.

The word "house" appears 9 times.
The word "classical" appears 8 times.
The word "progressive" appears 7 times.
The word "techno" appears 6 times.
The word "trance" appears 6 times.
The word "pop" appears 5 times.
The word "jazz" appears 5 times.
The word "industrial" appears 3 times.
The word "contemporary" appears 3 times.
The word "swedish" appears 3 times.
The word "rock" appears 3 times.
The word "electronic" appears 3 times.
The word "fol

One of the expected genres with the shortest tracks is indeed hardcore punk, but we can go in the opposite direction and find music for children too! The other results of more common genres (indie/rap/pop/beats/phonk) are possibly given by the emerging of artists producing tracks to be used in viral TikTok contents.
On the other hand, we have subgenres coming from the (relatively) new electronic music world (house/techno/trance/industrial) and from the (relatively) old world of jazz and classical music.

### Alternative version w/ unwind operator
We take into account all the (sub)genres each artist is related to; then, we take the average track lengths shorter than 2m15s and longer than 6m30s only.

In [24]:
query11_unwind = [
  {
    '$unwind': {
      'path': "$artist_genres"
    }
  }, 
  {
    '$group': {
      '_id': "$artist_genres",
      'averageTrackLengthPerGenre': {'$avg': "$duration_ms"}
    }
  }
  {
    '$project': {
      '_id': 0,
      'genre': "$_id",
      'averageTrackLengthPerGenre': 1
    }
  },
  {
    '$facet': {
        "shortest": [
            { '$sort': { "averageTrackLengthPerGenre": 1 } },
            { '$match': {"averageTrackLengthPerGenre": {"$lt": 135000}} }
        ],
        "longest": [
            { '$sort': { "averageTrackLengthPerGenre": -1 } },
            { '$match': {"averageTrackLengthPerGenre": {"$gt": 390000}} }
        ]
    }
  }
]

result_query11_unwind = list(all.aggregate(query11_unwind))

In [25]:
result_query11_unwind

[{'shortest': [{'averageTrackLengthPerGenre': 29568.317073170732,
    'genre': 'ringtone'},
   {'averageTrackLengthPerGenre': 60938.0, 'genre': 'vintage hollywood'},
   {'averageTrackLengthPerGenre': 61558.42857142857, 'genre': 'birthday'},
   {'averageTrackLengthPerGenre': 62262.4081632653, 'genre': 'sound effects'},
   {'averageTrackLengthPerGenre': 67542.23076923077, 'genre': 'slash punk'},
   {'averageTrackLengthPerGenre': 68000.0, 'genre': 'deep dance pop'},
   {'averageTrackLengthPerGenre': 74288.0, 'genre': 'russian viral rap'},
   {'averageTrackLengthPerGenre': 75696.0, 'genre': 'noise'},
   {'averageTrackLengthPerGenre': 75696.0, 'genre': 'japanoise'},
   {'averageTrackLengthPerGenre': 79004.0, 'genre': 'hypnagogic pop'},
   {'averageTrackLengthPerGenre': 80184.6, 'genre': 'plena uruguaya'},
   {'averageTrackLengthPerGenre': 82519.75, 'genre': 'ragga jungle'},
   {'averageTrackLengthPerGenre': 82726.1, 'genre': 'bachchon ke geet'},
   {'averageTrackLengthPerGenre': 84504.20408

In [26]:
shortest = [entry['shortest'] for entry in result_query11_unwind]
shortest = shortest[0]
longest = [entry['longest'] for entry in result_query11_unwind]
longest = longest[0]
shortest_genres = [entry['genre'] for entry in shortest]
longest_genres = [entry['genre'] for entry in longest]

# hardcore_count = 0
# classical_count = 0

# hardcore_count = sum(s.lower().split().count('hardcore') for s in shortest_genres)
# classical_count = sum(s.lower().split().count('classical') for s in longest_genres)

# print(hardcore_count)
# print(classical_count)

In [27]:
from collections import Counter

# Concatenate all strings into a single space-separated string
shortest_genres_text = ' '.join(shortest_genres)

# Split the combined string into a list of words
shortest_genres_words = shortest_genres_text.lower().split()

# Use Counter to count the frequency of each word
shortest_genres_word_frequency = Counter(shortest_genres_words)

# Print the words ordered by frequencies in descending order
for word, frequency in shortest_genres_word_frequency.most_common():
    if frequency >= 3:
        print(f'The word "{word}" appears {frequency} times.')
        
print('')

# Concatenate all strings into a single space-separated string
longest_genres_text = ' '.join(longest_genres)

# Split the combined string into a list of words
longest_genres_words = longest_genres_text.lower().split()

# Use Counter to count the frequency of each word
longest_genres_word_frequency = Counter(longest_genres_words)

# Print the words ordered by frequencies in descending order
for word, frequency in longest_genres_word_frequency.most_common():
    if frequency >= 3:
        print(f'The word "{word}" appears {frequency} times.')

# If you want to get the frequency of a specific word, you can do
# specified_word = "hardcore"
# specified_word_frequency = word_frequency[specified_word.lower()]
# print(f'The word "{specified_word}" appears {specified_word_frequency}

The word "rap" appears 8 times.
The word "classical" appears 6 times.
The word "punk" appears 5 times.
The word "indie" appears 5 times.
The word "hip" appears 5 times.
The word "hop" appears 5 times.
The word "russian" appears 4 times.
The word "musica" appears 4 times.
The word "rock" appears 4 times.
The word "japanese" appears 4 times.
The word "pop" appears 3 times.
The word "soundtrack" appears 3 times.
The word "hardcore" appears 3 times.
The word "beats" appears 3 times.
The word "lo-fi" appears 3 times.
The word "phonk" appears 3 times.
The word "kazakh" appears 3 times.
The word "jersey" appears 3 times.
The word "music"" appears 3 times.

The word "classical" appears 13 times.
The word "house" appears 13 times.
The word "techno" appears 13 times.
The word "rock" appears 10 times.
The word "progressive" appears 10 times.
The word "metal" appears 10 times.
The word "trance" appears 8 times.
The word "pop" appears 7 times.
The word "deep" appears 7 times.
The word "black" appea

We got similar results as before, with the exception of some classical subgenres that now enter in the picture of the shortest ones and the entrance of black metal in the longest ones.

# Query 12
## Multi-genre artists
Which will be the most fluid artists in the genres' space?

In [31]:
query12 = [
  {
    '$unwind': {
      'path': "$artist_genres"
    }
  }, 
  {
    '$group': {
      '_id': "$id",
      'artist_name': {"$first": "$name"},
      'genres_count': {'$sum': 1}
    }
  },
  {
    '$project': {
      '_id': 0,
      'artist_name': 1,
      'genres_count': 1
    }
  },
  { 
    '$sort': { 
      "genres_count": -1 
    } 
  }, 
  { 
    '$match': {
      "genres_count": {"$gte": 10}
    } 
  }
]

result_query12 = list(artists.aggregate(query12))

In [32]:
result_query12

[{'artist_name': 'The Moody Blues', 'genres_count': 14},
 {'artist_name': 'The Byrds', 'genres_count': 13},
 {'artist_name': 'Allman Brothers Band', 'genres_count': 13},
 {'artist_name': 'Derek & The Dominos', 'genres_count': 13},
 {'artist_name': 'Badfinger', 'genres_count': 12},
 {'artist_name': 'Procol Harum', 'genres_count': 12},
 {'artist_name': 'Emerson, Lake & Palmer', 'genres_count': 12},
 {'artist_name': 'Little Dragon', 'genres_count': 11},
 {'artist_name': 'The Yardbirds', 'genres_count': 11},
 {'artist_name': 'Grouper', 'genres_count': 11},
 {'artist_name': 'The Guess Who', 'genres_count': 11},
 {'artist_name': 'LCD Soundsystem', 'genres_count': 11},
 {'artist_name': 'Jimmy Eat World', 'genres_count': 11},
 {'artist_name': 'Robert Plant', 'genres_count': 10},
 {'artist_name': 'Heart', 'genres_count': 10},
 {'artist_name': 'Arcade Fire', 'genres_count': 10},
 {'artist_name': 'Jeff Beck', 'genres_count': 10},
 {'artist_name': 'Bad Company', 'genres_count': 10},
 {'artist_name

Seems like we have several artists mostly related to experimental genres (as well as rock subgenres)

Some examples: 

In [37]:
result_björk_query = list(artists.find({"name": "Björk"}, {"_id": 0, "name": 1, "artist_genres": 1}))
result_grouper_query = list(artists.find({"name": "Grouper"}, {"_id": 0, "name": 1, "artist_genres": 1}))
result_nin_query = list(artists.find({"name": "Nine Inch Nails"}, {"_id": 0, "name": 1, "artist_genres": 1}))

display_results(result_björk_query)
print('')
display_results(result_grouper_query)
print('')
display_results(result_nin_query)

{'name': 'Björk', 'artist_genres': ['art pop', 'electronica', 'experimental pop', 'experimental vocal', 'icelandic experimental', 'icelandic pop', 'icelandic singer-songwriter', 'metropopolis', 'permanent wave', 'trip hop']}

{'name': 'Grouper', 'artist_genres': ['ambient', 'ambient pop', 'art pop', 'dream pop', 'drone', 'electra', 'experimental ambient', 'experimental pop', 'modern dream pop', 'shoegaze', 'spectra']}

{'name': 'Nine Inch Nails', 'artist_genres': ['alternative metal', 'alternative rock', 'cyberpunk', 'electronic rock', 'industrial', 'industrial metal', 'industrial rock', 'nu metal', 'post-grunge', 'rock']}


# Query 13
## Number of subgenres
We've previously seen that exist 3000+ genres in our database, so how many (possibly useless) subgenres did Spotify create? \\
(Let's consider 5 main genres for our analysis: punk, jazz, classical, rock, pop)

In [42]:
query13 = [
  {
    '$unwind': {
      'path': "$artist_genres"
    }
  }, 
  {
    '$facet': {
        "punk_subgenres": [
            { '$match': {"artist_genres": {"$regex": "punk"}} },
            { '$group': { '_id': "$artist_genres"}},
            { '$project': { '_id': 0, "subgenre": "$_id"}}
        ],
        "punk_subgenres_count": [
            { '$match': {"artist_genres": {"$regex": "punk"}} },
            { '$group': { '_id': "$artist_genres", 
                          'duplicated_subgenres_count': {'$sum': 1}}},
            { '$group': { '_id': True, 
                          'unique_subgenres_count': {'$sum': 1}}},
            { '$project': { '_id': 0, 'unique_subgenres_count': 1}}
        ],
        "jazz_subgenres": [
            { '$match': {"artist_genres": {"$regex": "jazz"}} },
            { '$group': { '_id': "$artist_genres"}},
            { '$project': { '_id': 0, "subgenre": "$_id"}}
        ],
        "jazz_subgenres_count": [
            { '$match': {"artist_genres": {"$regex": "jazz"}} },
            { '$group': { '_id': "$artist_genres", 
                          'duplicated_subgenres_count': {'$sum': 1}}},
            { '$group': { '_id': True, 
                          'unique_subgenres_count': {'$sum': 1}}},
            { '$project': { '_id': 0, 'unique_subgenres_count': 1}}
        ],
        "classical_subgenres": [
            { '$match': {"artist_genres": {"$regex": "classical"}} },
            { '$group': { '_id': "$artist_genres"}},
            { '$project': { '_id': 0, "subgenre": "$_id"}}
        ],
        "classical_subgenres_count": [
            { '$match': {"artist_genres": {"$regex": "classical"}} },
            { '$group': { '_id': "$artist_genres", 
                          'duplicated_subgenres_count': {'$sum': 1}}},
            { '$group': { '_id': True, 
                          'unique_subgenres_count': {'$sum': 1}}},
            { '$project': { '_id': 0, 'unique_subgenres_count': 1}}
        ],
        "rock_subgenres": [
            { '$match': {"artist_genres": {"$regex": "rock"}} },
            { '$group': { '_id': "$artist_genres"}},
            { '$project': { '_id': 0, "subgenre": "$_id"}}
        ],
        "rock_subgenres_count": [
            { '$match': {"artist_genres": {"$regex": "rock"}} },
            { '$group': { '_id': "$artist_genres", 
                          'duplicated_subgenres_count': {'$sum': 1}}},
            { '$group': { '_id': True, 
                          'unique_subgenres_count': {'$sum': 1}}},
            { '$project': { '_id': 0, 'unique_subgenres_count': 1}}
        ],
        "pop_subgenres": [
            { '$match': {"artist_genres": {"$regex": "pop"}} },
            { '$group': { '_id': "$artist_genres"}},
            { '$project': { '_id': 0, "subgenre": "$_id"}}
        ],
        "pop_subgenres_count": [
            { '$match': {"artist_genres": {"$regex": "pop"}} },
            { '$group': { '_id': "$artist_genres", 
                          'duplicated_subgenres_count': {'$sum': 1}}},
            { '$group': { '_id': True, 
                          'unique_subgenres_count': {'$sum': 1}}},
            { '$project': { '_id': 0, 'unique_subgenres_count': 1}}
        ]
    }
  }
]

result_query13 = list(artists.aggregate(query13))

In [43]:
result_query13

[{'punk_subgenres': [{'subgenre': 'elektropunk'},
   {'subgenre': 'malang punk'},
   {'subgenre': 'uk post-punk revival'},
   {'subgenre': 'punk'},
   {'subgenre': 'icelandic punk'},
   {'subgenre': 'finnish pop punk'},
   {'subgenre': 'uk pop punk'},
   {'subgenre': 'pop punk'},
   {'subgenre': 'german punk'},
   {'subgenre': 'cowpunk'},
   {'subgenre': 'canadian punk'},
   {'subgenre': 'black punk'},
   {'subgenre': 'german punk rock'},
   {'subgenre': 'uk diy punk'},
   {'subgenre': 'emo punk'},
   {'subgenre': 'fast melodic punk'},
   {'subgenre': 'chinese punk'},
   {'subgenre': 'modern ska punk'},
   {'subgenre': 'early us punk'},
   {'subgenre': 'polish punk'},
   {'subgenre': 'deep hardcore punk'},
   {'subgenre': 'texas punk'},
   {'subgenre': 'new jersey punk'},
   {'subgenre': 'japanese punk rock'},
   {'subgenre': 'street punk'},
   {'subgenre': 'art punk'},
   {'subgenre': 'ska punk'},
   {'subgenre': 'punk cover'},
   {'subgenre': 'peruvian punk'},
   {'subgenre': 'irish 

It is clear that the presence of the majority of the subgenres is due to geographic aspects, which may be a not so meaningful way to make reasonable distictions between very similar genres (e.g. "hyperpop italiano/japanese hyperpop").